In [20]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
next(model.parameters()).device


True
Tesla T4


device(type='cuda', index=0)

In [21]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from transformers import BertModel, BertForSequenceClassification, BertTokenizer, TrainingArguments, Trainer
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [22]:
data = pd.read_csv('train_en.txt', sep='\t')

In [23]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [24]:
sentences = data['Sentence'].values.tolist()


In [25]:
encodings = tokenizer(
    sentences,
    padding=True,
    truncation=True,
    max_length=128
)

In [26]:
encodings['attention_mask']

[[1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,

In [27]:
labels = [1 if label=='toxic' else 0 for label in data['Style'].values]


In [28]:
class ToxicDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [29]:
dataset = ToxicDataset(encodings, labels)


In [30]:
model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=2
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
training_args = TrainingArguments(output_dir='toxic_bert', report_to='none',     per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.005,
    optim="adamw_torch",

    logging_steps=2000,
    save_strategy="no",

    fp16=True)

In [32]:
trainer1 = Trainer(model=model, args=training_args, train_dataset=dataset, tokenizer=tokenizer)

/tmp/ipython-input-4004829264.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer1 = Trainer(model=model, args=training_args, train_dataset=dataset, tokenizer=tokenizer)


In [33]:
trainer1.train()

Step,Training Loss
2000,0.002700
4000,0.000000
6000,0.000000
8000,0.000000
10000,0.000000
12000,0.000000
14000,0.000000
16000,0.000000
18000,0.000000
20000,0.000000


TrainOutput(global_step=128592, training_loss=4.2113785594757386e-05, metrics={'train_runtime': 13153.0319, 'train_samples_per_second': 78.211, 'train_steps_per_second': 9.777, 'total_flos': 6.76665723286656e+16, 'train_loss': 4.2113785594757386e-05, 'epoch': 3.0})

In [34]:
test_data = pd.read_csv("test_en.txt", sep="\t")
test_sentences = test_data["Sentence"].values.tolist()
test_encodings = tokenizer(
    test_sentences,
    padding=True,
    truncation=True,
    max_length=128
)

In [35]:
test_labels = [1 if lbl == "toxic" else 0 for lbl in test_data["Style"].values]

test_dataset = ToxicDataset(test_encodings, test_labels)

In [36]:
test_results1 = trainer1.predict(test_dataset)
print("MODEL 1 TEST RESULTS:", test_results1.metrics)

MODEL 1 TEST RESULTS: {'test_loss': 0.0, 'test_runtime': 95.4085, 'test_samples_per_second': 449.268, 'test_steps_per_second': 56.159}


----------------------------------------------MODEL 2----------------------------------------------

In [37]:
model2 = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=2
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
training_args2 = TrainingArguments(
    output_dir="roberta_base_model2",
    report_to="none",
    learning_rate=1e-5,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    weight_decay=0.0,
    save_strategy="no"
)

In [39]:
trainer2 = Trainer(
    model=model2,
    args=training_args2,
    train_dataset=dataset,
    tokenizer=tokenizer
)

/tmp/ipython-input-3865317231.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer2 = Trainer(


In [ ]:
trainer2.train()

Step,Training Loss
500,0.019000
1000,0.000100
1500,0.000000
2000,0.000000
2500,0.000000
3000,0.000000
3500,0.000000
4000,0.000000
4500,0.000000
5000,0.000000


In [ ]:
test_results2 = trainer2.predict(test_dataset)
print("MODEL 2 TEST RESULTS:", test_results2.metrics)

In [ ]:
model3 = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=2
)

In [ ]:
training_args3 = TrainingArguments(
    output_dir="roberta_base_model3",
    report_to="none",
    learning_rate=2e-5,
    num_train_epochs=6,
    per_device_train_batch_size=16,
    weight_decay=0.01,
    save_strategy="no"
)


In [ ]:
trainer3 = Trainer(
    model=model3,
    args=training_args3,
    train_dataset=dataset,
    tokenizer=tokenizer
)

In [ ]:
trainer3.train()

In [ ]:
test_results3 = trainer3.predict(test_dataset)
print("MODEL 3 TEST RESULTS:", test_results3.metrics)

# TASK 2

In [ ]:
import pandas as pd
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
from sentence_transformers import SentenceTransformer, util

In [ ]:
model1_name = 't5-base'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
model1 = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
data = pd.read_csv('test_en.txt', sep='\t')
sentences = data['Sentence'].values.tolist()
labels = data['Style'].values.tolist()

In [ ]:
sample_sentence = sentences[4]
sample_sentence

In [ ]:
prompt = f'Classify the following text into either \'TOXIC\' or \'NEUTRAL\': {sample_sentence}'
prompt

In [ ]:
tokens = tokenizer(prompt, return_tensors='pt')

In [ ]:
output_ids = model.generate(tokens.input_ids)

In [ ]:
output_ids = model.generate(tokens.input_ids)
output_ids

In [ ]:
examples = 'Text: delete the page and shut up\nClass: TOXIC\nText: I heard it was on the news.\nClass: NEUTRAL\n'
examples

In [ ]:
prompt = f'{examples}\nClassify the following text into either \'TOXIC\' or \'NEUTRAL\': {sample_sentence}\nClass:'
prompt

In [ ]:
embedding_model = SentenceTransformer('all-distilroberta-v1')
embeddings = embedding_model.encode(sentences[:20], batch_size=64, show_progress_bar=True)

In [ ]:
embeddings.shape

In [ ]:
query_emb = embedding_model.encode([sample_sentence], batch_size=64, show_progress_bar=True)

In [ ]:
util.semantic_search(query_emb, embeddings, top_k=10)

In [ ]:
t5_name = "google/flan-t5-base"
bart_name = "facebook/bart-large"

In [ ]:
t5_tokenizer = AutoTokenizer.from_pretrained(t5_name)
t5_model = AutoModelForSeq2SeqLM.from_pretrained(t5_name)

In [ ]:
bart_tokenizer = AutoTokenizer.from_pretrained(bart_name)
bart_model = AutoModelForSeq2SeqLM.from_pretrained(bart_name)

In [ ]:
def classify_t5_zeroshot(text):
    prompt = f"Is the following review positive or negative?\nReview: \"{text}\"\nAnswer:"
    inputs = t5_tokenizer(prompt, return_tensors="pt")
    outputs = t5_model.generate(**inputs, max_new_tokens=5)
    answer = t5_tokenizer.decode(outputs[0], skip_special_tokens=True).lower()

    return 1 if "positive" in answer else 0


In [ ]:
def classify_bart_zeroshot(text):
    prompt = f"Is the following review positive or negative?\nReview: \"{text}\"\nAnswer:"
    inputs = bart_tokenizer(prompt, return_tensors="pt")
    outputs = bart_model.generate(**inputs, max_new_tokens=5)
    answer = bart_tokenizer.decode(outputs[0], skip_special_tokens=True).lower()

    return 1 if "positive" in answer else 0


In [ ]:
pred_t5_zero = [classify_t5_zeroshot(t) for t in tqdm(texts)]
pred_bart_zero = [classify_bart_zeroshot(t) for t in tqdm(texts)]


In [ ]:
def evaluate(name, preds):
    print(f"\n=== {name} ===")
    print("Accuracy:", accuracy_score(true_labels, preds))
    print("Precision:", precision_score(true_labels, preds))
    print("Recall:", recall_score(true_labels, preds))
    print("F1:", f1_score(true_labels, preds))


In [ ]:
evaluate("T5 Zero-Shot", pred_t5_zero)
evaluate("BART Zero-Shot", pred_bart_zero)


In [ ]:
fewshot_examples = """
Review: "The rice was very flavorful and delicious. I went back for more! yum! "
Sentiment: positive

Review: "This place won't last."
Sentiment: negative
"""

def classify_t5_fewshot(text):
    prompt = fewshot_examples + f'\nReview: "{text}"\nSentiment: '
    inputs = t5_tokenizer(prompt, return_tensors="pt")
    outputs = t5_model.generate(**inputs, max_new_tokens=5)
    answer = t5_tokenizer.decode(outputs[0], skip_special_tokens=True).lower()
    return 1 if "positive" in answer else 0


### Step 1: Move Models to GPU

First, let's make sure our T5 and BART models are loaded onto the GPU if available. This can significantly speed up inference.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

t5_model.to(device)
bart_model.to(device)

### Step 2: Implement Batching for Inference

Instead of processing each sentence individually, we can group them into batches. This allows the GPU to process multiple inputs in parallel.

Here's an example of how you can modify the `classify_t5_zeroshot` function to accept a list of texts and process them in batches. We'll also ensure the input tensors are moved to the GPU.

In [ ]:
from tqdm.auto import tqdm # Import tqdm for progress bar

def classify_t5_zeroshot_batched(texts, batch_size=8):
    predictions = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Classifying with T5 (Batched)"):
        batch_texts = texts[i:i + batch_size]

        # Prepare prompt for each text in the batch
        prompts = [f"Is the following review positive or negative?\nReview: \"{text}\"\nAnswer:" for text in batch_texts]

        inputs = t5_tokenizer(prompts, return_tensors="pt", padding=True, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()} # Move inputs to GPU

        outputs = t5_model.generate(**inputs, max_new_tokens=5)
        decoded_outputs = t5_tokenizer.batch_decode(outputs, skip_special_tokens=True)

        batch_predictions = [1 if "positive" in answer.lower() else 0 for answer in decoded_outputs]
        predictions.extend(batch_predictions)

    return predictions

def classify_bart_zeroshot_batched(texts, batch_size=8):
    predictions = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Classifying with BART (Batched)"):
        batch_texts = texts[i:i + batch_size]

        prompts = [f"Is the following review positive or negative?\nReview: \"{text}\"\nAnswer:" for text in batch_texts]

        inputs = bart_tokenizer(prompts, return_tensors="pt", padding=True, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()} # Move inputs to GPU

        outputs = bart_model.generate(**inputs, max_new_tokens=5)
        decoded_outputs = bart_tokenizer.batch_decode(outputs, skip_special_tokens=True)

        batch_predictions = [1 if "positive" in answer.lower() else 0 for answer in decoded_outputs]
        predictions.extend(batch_predictions)

    return predictions

Now you would call these batched functions with your `texts` list (assuming `texts` and `true_labels` are defined from a previous step) and then evaluate them. Make sure `true_labels` is also available for evaluation. For example, using the `test_data` loaded earlier:

In [ ]:
# Assuming 'test_data' was loaded earlier with sentences and labels
texts = test_data['Sentence'].values.tolist()
true_labels = [1 if label == 'positive' else 0 for label in test_data['Style'].values]

pred_t5_zero_batched = classify_t5_zeroshot_batched(texts)
pred_bart_zero_batched = classify_bart_zeroshot_batched(texts)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

evaluate("T5 Zero-Shot (Batched)", pred_t5_zero_batched)
evaluate("BART Zero-Shot (Batched)", pred_bart_zero_batched)